In [ ]:
from __future__ import print_function
import numpy as np
# import matplotlib.pyplot as plt
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from itertools import cycle
import matplotlib.pyplot as plt
import os
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as tfunc
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as func
from torchvision import models
from timeit import default_timer as timer
import time
from GCE import *
from advertorch.attacks import  GradientSignAttack,LinfPGDAttack
import copy

In [ ]:
datadir = 'data/'
traindir = datadir + 'train/'
testdir = datadir + 'test/'
valdir = datadir + 'val/'
unlabeldir = datadir + 'unlabel/'

In [ ]:
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor()]),
    'unlabel':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor()]),
    'val':
        transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor()]),
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
    ]),
}

In [ ]:
batch_size = 64
data = {'train':datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
        'unlabel':datasets.ImageFolder(root=unlabeldir, transform=image_transforms['unlabel']),
        'val':datasets.ImageFolder(root=valdir, transform=image_transforms['val'])}
        
# Dataloader iterators
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True, num_workers =8, pin_memory = True),
    'unlabel': DataLoader(data['unlabel'], batch_size=batch_size, shuffle=True, num_workers =8, pin_memory = True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=False,num_workers =8,pin_memory = True)}

In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
input_size = 224
model.cuda();

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
#criterion = GuidedComplementEntropy(0.333)

In [ ]:
device = torch.device("cuda")

In [ ]:
num_epochs = 10

In [ ]:
def val(model):
    model.eval()
    correct = 0
    size = 0 
    for tedata, tetarget in dataloaders['val']:
        size += tedata.shape[0]
        tedata, tetarget = tedata.to(device), tetarget.to(device)

        output = model(tedata)
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(tetarget.view_as(pred)).sum().item()

    print("{:s} acc: {:.2f}".format('clean', 100. * correct / size)) 

In [ ]:
since = time.time()

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    #copy_weights

    model.train()  # Set model to training mode
    running_loss = 0.0
    running_corrects = 0

        # Iterate over data.
    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # eps_now= np.random.uniform(0.001,0.003)
        # FGSM = GradientSignAttack(model, loss_fn=nn.CrossEntropyLoss(reduction="sum"),eps=eps_now)
        # advdata = FGSM.perturb(inputs, labels).to(device)
        # inputs= torch.cat((inputs, advdata),0)
        # labels= torch.cat((labels, labels))

        #generate attacks
        iterator = iter(dataloaders['unlabel'])
        inputs_u,labels_u = iterator.next()
        inputs_u = inputs_u.to(device)
        labels_u = labels_u.to(device)

        eps_now= np.random.uniform(0.001,0.003)
        FGSM = GradientSignAttack(model, loss_fn=nn.CrossEntropyLoss(reduction="sum"),eps=eps_now)
        inputs_u_adv = FGSM.perturb(inputs_u).to(device)
        outputs_adv = model(inputs_u_adv)
        _,labels_pseudo = torch.max(model(inputs_u),1)

        
        # zero the parameter gradients
        optimizer.zero_grad()
        outputs = model(inputs)
#        softmax
        loss1 = criterion(outputs, labels)
        loss2 = criterion(outputs_adv, labels_pseudo)
        loss = loss1 +3*loss2
        #loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        loss.backward()
        optimizer.step()

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloaders['train'].dataset)
    epoch_acc = running_corrects.double() / len(dataloaders['train'].dataset)

    print('{} Loss: {:.4f} Acc: {:.4f}'.format('train', epoch_loss, epoch_acc))
    val(model)
    print()

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

In [ ]:
model.eval()
for i in range(11):
    correct = 0
    size = 0
    n = 0
    for tedata, tetarget in dataloaders['test']:
        size += tedata.shape[0]
        tedata, tetarget = tedata.to(device), tetarget.to(device)

        
        eps_now = i*0.0002
        pgd = GradientSignAttack(model, loss_fn=nn.CrossEntropyLoss(reduction="sum"),eps=eps_now)
        tedata = pgd.perturb(tedata, tetarget).to(device)
        output = model(tedata)
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(tetarget.view_as(pred)).sum().item()

    print("{:f} acc: {:.2f}".format(i, 100. * correct / size)) 